## Takuro Sazaki
## UNI: ts3422
## Github repo: https://github.com/tzak16/QMSS-G5074.git

# Instructions:

Your final report should be written up in a Jupyter notebook.  It should be posted to a public Github repo as an ipynb  submitted to this assignment via courseworks.  Please include the link to your Github repo in this ipynb file.

Use the deep learning and sklearn example ipynb notebooks from the Week 11 folder for example submission code.

Note: After installing aimodelshare in Colab you may need use dropdown menus to restart yours session before running code.  Use Runtime > restart runtime to restart your Colab session.

Your report should include the following information:

## Preprocessing

In [ ]:
#install aimodelshare library
! pip install aimodelshare==0.0.189

In [ ]:
! pip install protobuf==3.19.6

In [3]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/sst2_competition_data-repository:latest') 


Data downloaded successfully.


In [17]:
# Set up X_train, X_test, and y_train_labels objects
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

X_train=pd.read_csv("sst2_competition_data/X_train.csv", squeeze=True)
X_test=pd.read_csv("sst2_competition_data/X_test.csv", squeeze=True)

y_train_labels=pd.read_csv("sst2_competition_data/y_train_labels.csv", squeeze=True)

# ohe encode Y data
y_train = pd.get_dummies(y_train_labels)

X_train.head()

0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
2    Singer/composer Bryan Adams contributes a slew...
3                 Yet the act is still charming here .
4    Whether or not you 're enlightened by any of D...
Name: text, dtype: object

In [18]:
# This preprocessor function makes use of the tf.keras tokenizer

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 40)
(1821, 40)


### Discuss the dataset in general terms and describe why building a predictive model using this data might be practically useful.  Who could benefit from a model like this? Explain.

he dataset comprises 50,000 movie reviews, which are divided into training and testing sets and labeled with a binary sentiment classification - either "positive" or "negative". <br/>Developing a predictive model using this data can be highly beneficial as it enables us to classify the sentiment of movie reviews without requiring human intervention. This can be particularly useful for film production companies that seek to analyze and aggregate movie reviews.

### Run at least three prediction models to try to predict the SST sentiment dataset well. 

#### Use an Embedding layer and LSTM layers in at least one model

In [134]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

model1 = Sequential()
model1.add(Embedding(10000, 16, input_length=40))
model1.add(LSTM(32, return_sequences=True))
model1.add(LSTM(32, return_sequences=True))
model1.add(LSTM(32, return_sequences=True))
model1.add(LSTM(32))
model1.add(Flatten())
model1.add(Dense(2, activation='softmax'))

model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model1.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 22s 86ms/step - loss: 0.6484 - acc: 0.6306 - val_loss: 0.7971 - val_acc: 0.4299
Epoch 2/10
173/173 [==============================] - 14s 81ms/step - loss: 0.4802 - acc: 0.7720 - val_loss: 0.7037 - val_acc: 0.6611
Epoch 3/10
173/173 [==============================] - 13s 75ms/step - loss: 0.3578 - acc: 0.8483 - val_loss: 0.8124 - val_acc: 0.5571
Epoch 4/10
173/173 [==============================] - 11s 66ms/step - loss: 0.2859 - acc: 0.8853 - val_loss: 0.5272 - val_acc: 0.7536
Epoch 5/10
173/173 [==============================] - 14s 81ms/step - loss: 0.2259 - acc: 0.9093 - val_loss: 0.7685 - val_acc: 0.6568
Epoch 6/10
173/173 [==============================] - 13s 74ms/step - loss: 0.1873 - acc: 0.9245 - val_loss: 0.7220 - val_acc: 0.7276
Epoch 7/10
173/173 [==============================] - 12s 68ms/step - loss: 0.1611 - acc: 0.9330 - val_loss: 0.7544 - val_acc: 0.7413
Epoch 8/10
173/173 [==============================] - 14s 80ms

In [20]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


In [135]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features are there?
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model1 = model_to_onnx(model1, framework='keras',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=True)

with open("model1.onnx", "wb") as f:
    f.write(onnx_model1.SerializeToString())

In [27]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://rlxjxnoql9.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this Movie Review Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [28]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [137]:
#Submit Model 1: 

#-- Generate predicted values (a list of predicted labels "positive" or "negative") (Model 1)
prediction_column_index = model1.predict(preprocessor(X_test)).argmax(axis=1)
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model1.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                                 custom_metadata = {"team": "3"})

57/57 [==============================] - 3s 20ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 272

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

#### Use an Embedding layer and Conv1d layers in at least one model

In [139]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

model2 = Sequential()
model2.add(Embedding(10000, 16, input_length=40))
model2.add(layers.Conv1D(32, 3, activation='relu')) 
model2.add(layers.Conv1D(32, 3, activation='relu')) 
model2.add(layers.MaxPooling1D(3))
model2.add(layers.Conv1D(32, 3, activation='relu')) 
model2.add(layers.Conv1D(32, 3, activation='relu')) 
model2.add(layers.GlobalAveragePooling1D())
model2.add(Dense(2, activation='softmax'))

model2.summary()

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_38 (Embedding)    (None, 40, 16)            160000    
                                                                 
 conv1d_109 (Conv1D)         (None, 38, 32)            1568      
                                                                 
 conv1d_110 (Conv1D)         (None, 36, 32)            3104      
                                                                 
 max_pooling1d_30 (MaxPoolin  (None, 12, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_111 (Conv1D)         (None, 10, 32)            3104      
                                                                 
 conv1d_112 (Conv1D)         (None, 8, 32)             3104      
                                                     

In [140]:
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model2.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 3s 9ms/step - loss: 0.6648 - acc: 0.6134 - val_loss: 0.9901 - val_acc: 0.1488
Epoch 2/10
173/173 [==============================] - 1s 8ms/step - loss: 0.5598 - acc: 0.6954 - val_loss: 0.7181 - val_acc: 0.6142
Epoch 3/10
173/173 [==============================] - 1s 8ms/step - loss: 0.4034 - acc: 0.8228 - val_loss: 0.4946 - val_acc: 0.7796
Epoch 4/10
173/173 [==============================] - 1s 8ms/step - loss: 0.3148 - acc: 0.8701 - val_loss: 0.5110 - val_acc: 0.7695
Epoch 5/10
173/173 [==============================] - 1s 8ms/step - loss: 0.2561 - acc: 0.8951 - val_loss: 0.6079 - val_acc: 0.7572
Epoch 6/10
173/173 [==============================] - 1s 8ms/step - loss: 0.2088 - acc: 0.9164 - val_loss: 0.6767 - val_acc: 0.7421
Epoch 7/10
173/173 [==============================] - 2s 13ms/step - loss: 0.1742 - acc: 0.9301 - val_loss: 0.6950 - val_acc: 0.7616
Epoch 8/10
173/173 [==============================] - 2s 13ms/step - loss: 

In [141]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features are there?
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model2 = model_to_onnx(model2, framework='keras',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model2.SerializeToString())

In [142]:
#Submit Model 2: 

#-- Generate predicted values (a list of predicted labels "positive" or "negative") (Model 1)
prediction_column_index = model2.predict(preprocessor(X_test)).argmax(axis=1)
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                                 custom_metadata = {"team": "3"})

57/57 [==============================] - 0s 4ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 273

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

#### Use transfer learning with glove embeddings for at least one of these models

In [75]:
# What if we wanted to use a matrix of pretrained embeddings?  Same as transfer learning before, but now we are importing a pretrained Embedding matrix:
# Download Glove embedding matrix weights (Might take 10 mins or so!)
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2023-04-17 00:23:56--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-04-17 00:23:56--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-04-17 00:23:56--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [76]:
! unzip glove.6B.zip 

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [78]:
# Extract embedding data for 100 feature embedding matrix
import os
glove_dir = os.getcwd()

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [87]:
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index

In [88]:
# Build embedding matrix
embedding_dim = 100 # change if you use txt files using larger number of features
maxlen = 40  # We will cut reviews after 100 words in sequence
max_words = 10000  # We will only consider the top 10,000 words in the dataset

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [145]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

model2 = Sequential()
model2.add(layers.Embedding(max_words, embedding_dim, input_length=maxlen))
model2.add(layers.Conv1D(32, 3, activation='relu')) 
model2.add(layers.Conv1D(32, 3, activation='relu')) 
model2.add(layers.MaxPooling1D(3))
model2.add(layers.Conv1D(32, 3, activation='relu')) 
model2.add(layers.Conv1D(32, 3, activation='relu')) 
model2.add(layers.GlobalMaxPooling1D())
model2.add(Dense(2, activation='softmax'))

In [146]:
model2.layers[0].set_weights([embedding_matrix])
model2.layers[0].trainable = False

model2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model2.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)
model2.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
173/173 [==============================] - 3s 10ms/step - loss: 0.6223 - acc: 0.6434 - val_loss: 0.6924 - val_acc: 0.6236
Epoch 2/10
173/173 [==============================] - 2s 9ms/step - loss: 0.5406 - acc: 0.7227 - val_loss: 0.6035 - val_acc: 0.7168
Epoch 3/10
173/173 [==============================] - 2s 9ms/step - loss: 0.4968 - acc: 0.7542 - val_loss: 0.5931 - val_acc: 0.7182
Epoch 4/10
173/173 [==============================] - 2s 9ms/step - loss: 0.4688 - acc: 0.7726 - val_loss: 0.5515 - val_acc: 0.7478
Epoch 5/10
173/173 [==============================] - 2s 12ms/step - loss: 0.4413 - acc: 0.7870 - val_loss: 0.7053 - val_acc: 0.6416
Epoch 6/10
173/173 [==============================] - 3s 15ms/step - loss: 0.4088 - acc: 0.8083 - val_loss: 0.5969 - val_acc: 0.7103
Epoch 7/10
173/173 [==============================] - 3s 15ms/step - loss: 0.3778 - acc: 0.8273 - val_loss: 0.7637 - val_acc: 0.6380
Epoch 8/10
173/173 [==============================] - 3s 15ms/step - los

In [147]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features are there?
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model3 = model_to_onnx(model2, framework='keras',
                          initial_types=initial_type,
                          transfer_learning=True,
                          deep_learning=True)

with open("model3.onnx", "wb") as f:
    f.write(onnx_model3.SerializeToString())

In [148]:
#Submit Model 3: 

#-- Generate predicted values (a list of predicted labels "positive" or "negative") (Model 1)
prediction_column_index = model2.predict(preprocessor(X_test)).argmax(axis=1)
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 3 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model3.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                                 custom_metadata = {"team": "3"})

57/57 [==============================] - 0s 3ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 274

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

#### Discuss which models performed better and point out relevant hyper-parameter values for successful models.

Deep learning model with four LSTM layers performed the best among three models I tried. I put the units of 32 for all of the four LSTM layers.
<br/> Transfer learning improved the performance of Conv1D model, but the performance did not exceed that of the LSTM model.

#### Submit your best three models to the leader board for the SST Model Share competition.

### After you submit your first three models, describe your best model with your team via your team slack channel 

#### Fit and submit up to three more models after learning from your team.

##### 1. Transfer learning with CNN

In [156]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

model4 = Sequential()
model4.add(layers.Embedding(max_words, embedding_dim, input_length=maxlen))
model4.add(layers.Conv1D(16, 6, activation='relu')) 
model4.add(layers.Conv1D(16, 6, activation='relu')) 
model4.add(layers.Conv1D(16, 3, activation='relu')) 
model4.add(layers.Conv1D(16, 3, activation='relu')) 
model4.add(layers.GlobalMaxPooling1D())
model4.add(Dense(2, activation='softmax'))

In [157]:
model4.layers[0].set_weights([embedding_matrix])
model4.layers[0].trainable = False

model4.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model4.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)
model4.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
173/173 [==============================] - 4s 16ms/step - loss: 0.6331 - acc: 0.6371 - val_loss: 0.8716 - val_acc: 0.3613
Epoch 2/10
173/173 [==============================] - 2s 14ms/step - loss: 0.5581 - acc: 0.7029 - val_loss: 0.9604 - val_acc: 0.4436
Epoch 3/10
173/173 [==============================] - 3s 15ms/step - loss: 0.5224 - acc: 0.7365 - val_loss: 0.5930 - val_acc: 0.7290
Epoch 4/10
173/173 [==============================] - 2s 13ms/step - loss: 0.4949 - acc: 0.7509 - val_loss: 0.6509 - val_acc: 0.6929
Epoch 5/10
173/173 [==============================] - 2s 10ms/step - loss: 0.4706 - acc: 0.7751 - val_loss: 0.6186 - val_acc: 0.7175
Epoch 6/10
173/173 [==============================] - 3s 15ms/step - loss: 0.4440 - acc: 0.7861 - val_loss: 1.0402 - val_acc: 0.4574
Epoch 7/10
173/173 [==============================] - 3s 19ms/step - loss: 0.4142 - acc: 0.8094 - val_loss: 0.5731 - val_acc: 0.7399
Epoch 8/10
173/173 [==============================] - 4s 21ms/step - 

In [158]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features are there?
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model4 = model_to_onnx(model4, framework='keras',
                          initial_types=initial_type,
                          transfer_learning=True,
                          deep_learning=True)

with open("model4.onnx", "wb") as f:
    f.write(onnx_model3.SerializeToString())

In [159]:
#Submit Model 4: 

#-- Generate predicted values (a list of predicted labels "positive" or "negative") (Model 1)
prediction_column_index = model4.predict(preprocessor(X_test)).argmax(axis=1)
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 4 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model4.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                                 custom_metadata = {"team": "3"})

57/57 [==============================] - 0s 5ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 275

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

##### 2. Transfer learning with SLTM

In [161]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten

model5 = Sequential()
model5.add(layers.Embedding(max_words, embedding_dim, input_length=maxlen))
model5.add(LSTM(32, return_sequences=True))
model5.add(LSTM(32, return_sequences=True))
model5.add(LSTM(32, return_sequences=True))
model5.add(LSTM(32))
model5.add(Flatten())
model5.add(Dense(32, activation='softmax'))
model5.add(Dense(16, activation='softmax'))
model5.add(Dense(2, activation='softmax'))

In [162]:
model5.layers[0].set_weights([embedding_matrix])
model5.layers[0].trainable = False

model5.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model5.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 23s 78ms/step - loss: 0.6757 - acc: 0.6149 - val_loss: 0.8189 - val_acc: 0.1488
Epoch 2/10
173/173 [==============================] - 13s 73ms/step - loss: 0.6677 - acc: 0.6149 - val_loss: 0.8568 - val_acc: 0.1488
Epoch 3/10
173/173 [==============================] - 15s 87ms/step - loss: 0.6668 - acc: 0.6149 - val_loss: 0.8765 - val_acc: 0.1488
Epoch 4/10
173/173 [==============================] - 22s 128ms/step - loss: 0.6666 - acc: 0.6149 - val_loss: 0.8843 - val_acc: 0.1488
Epoch 5/10
173/173 [==============================] - 13s 76ms/step - loss: 0.6666 - acc: 0.6149 - val_loss: 0.8798 - val_acc: 0.1488
Epoch 6/10
173/173 [==============================] - 15s 89ms/step - loss: 0.6666 - acc: 0.6149 - val_loss: 0.8829 - val_acc: 0.1488
Epoch 7/10
173/173 [==============================] - 14s 80ms/step - loss: 0.6666 - acc: 0.6149 - val_loss: 0.8820 - val_acc: 0.1488
Epoch 8/10
173/173 [==============================] - 14s 79m

In [164]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features are there?
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model5 = model_to_onnx(model5, framework='keras',
                          initial_types=initial_type,
                          transfer_learning=False,
                          deep_learning=True)

with open("model5.onnx", "wb") as f:
    f.write(onnx_model5.SerializeToString())

In [165]:
#Submit Model 5: 

#-- Generate predicted values (a list of predicted labels "positive" or "negative") (Model 1)
prediction_column_index = model5.predict(preprocessor(X_test)).argmax(axis=1)
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 5 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model5.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                                 custom_metadata = {"team": "3"})

57/57 [==============================] - 4s 29ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 278

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

##### 3. Transfer learning with CNN, using Keras Tuner

In [171]:
! pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.1 MB/s eta 0:00:00


In [178]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
from tensorflow import keras
import keras_tuner as kt

#Define model structure & parameter search space with function
def build_model(hp):
    model = keras.Sequential()
    model.add(Embedding(10000, 100, input_length=40))
    model.add(layers.Conv1D(filters=hp.Int("filters", min_value=8, max_value=32, step=4), kernel_size=3, strides=3))
    model.add(layers.Conv1D(filters=hp.Int("filters", min_value=8, max_value=32, step=4), kernel_size=3, strides=3))
    model.add(layers.Conv1D(filters=hp.Int("filters", min_value=8, max_value=32, step=4), kernel_size=3, strides=3))
    model.add(layers.GlobalMaxPooling1D())
    model.add(Dense(2, activation='softmax'))
    model.compile(
        optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"],
    )
    return model

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=3, #max number of trials to run during search
    executions_per_trial=1, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(preprocessor(X_train), y_train, epochs=1, validation_split=0.2)

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.1488439291715622

Best val_accuracy So Far: 0.1488439291715622
Total elapsed time: 00h 00m 16s


In [182]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])

In [181]:
tuned_model.layers[0].set_weights([embedding_matrix])
tuned_model.layers[0].trainable = False

tuned_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = tuned_model.fit(preprocessor(X_train), y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)
tuned_model.save_weights('pre_trained_glove_model.h5')

Epoch 1/10
173/173 [==============================] - 2s 6ms/step - loss: 0.6672 - acc: 0.6102 - val_loss: 0.9356 - val_acc: 0.1655
Epoch 2/10
173/173 [==============================] - 1s 6ms/step - loss: 0.6410 - acc: 0.6295 - val_loss: 0.8749 - val_acc: 0.2478
Epoch 3/10
173/173 [==============================] - 1s 6ms/step - loss: 0.6251 - acc: 0.6454 - val_loss: 0.8897 - val_acc: 0.2847
Epoch 4/10
173/173 [==============================] - 1s 7ms/step - loss: 0.6116 - acc: 0.6570 - val_loss: 0.8641 - val_acc: 0.3049
Epoch 5/10
173/173 [==============================] - 1s 7ms/step - loss: 0.6022 - acc: 0.6660 - val_loss: 0.7972 - val_acc: 0.3642
Epoch 6/10
173/173 [==============================] - 1s 8ms/step - loss: 0.5921 - acc: 0.6729 - val_loss: 0.8043 - val_acc: 0.3822
Epoch 7/10
173/173 [==============================] - 1s 7ms/step - loss: 0.5847 - acc: 0.6801 - val_loss: 0.8396 - val_acc: 0.3519
Epoch 8/10
173/173 [==============================] - 1s 8ms/step - loss: 0.

In [183]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features are there?
from skl2onnx.common.data_types import FloatTensorType

feature_count=preprocessor(X_test).shape[1] #Get count of preprocessed features
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model6 = model_to_onnx(tuned_model, framework='keras',
                          initial_types=initial_type,
                          transfer_learning=True,
                          deep_learning=True)

with open("model6.onnx", "wb") as f:
    f.write(onnx_model6.SerializeToString())

In [184]:
#Submit Model 6: 

#-- Generate predicted values (a list of predicted labels "positive" or "negative") (Model 1)
prediction_column_index = tuned_model.predict(preprocessor(X_test)).argmax(axis=1)
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 6 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model6.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels,
                                 custom_metadata = {"team": "3"})

57/57 [==============================] - 0s 2ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 279

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [ ]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

#### Discuss results

(Conv1D models)
I tried keras_tune to find the best value for the number of filters, but this attempt yielded a poor result. I also decreased the number of filters and reduced the kernel size, while adding more Conv1D layers, but it did not perform well.

<br/>
(LSTM models)
Adding multiple Dense layers deteriorated the performance of the model.

### Discuss which models you tried and which models performed better and point out relevant hyper-parameter values for successful models.

I tried six models in total.

*   Embedding layer and four LSTM layers and one Dense layer, no dropout, 10 epochs --- accuracy = 80.02%
*   Embedding layer and two Conv1d layers + MaxPooling + two Conv1d layers + GlobalAveragePooling --- accuracy = 75.74%
*   Transfer learning with glove embeddings on the second model --- accuracy = 77.61%
*   Transfer learning with glove embeddings, four Conv1D with 16 filters and kernel size of 3 --- accuracy = 73.22%
*   Transfer learning with glove embeddings, added two Dense layers --- accuracy = 49.95%
*   Transfer learning with glove embeddings, used Keras_tune --- accuracy = 50.60%

<br/>
The most successful model was Embedding layer and four LSTM layers and one Dense layer, no dropout, 10 epochs. Inserting glove embeddings did not necessarily improve the results. Changing from MaxPooling to AveragePooling improved the accuracy.